# Data Description

### import libraries

In [16]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import os

In [38]:
folder_path = "/Users/yusuke_sugihara/DA_Project/07_news_Channels/20news-bydate-train/sci.space/"
file_names = os.listdir(folder_path)
print(len(file_names))


593
['60923', '61039', '60782', '61234', '61006']
